In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import re
import nltk
import sys  
!{sys.executable} -m pip install contractions
from contractions import contractions_dict
from nltk.corpus import stopwords
!{sys.executable} -m pip install  pip install spacy
from spacy.lang.en.stop_words import STOP_WORDS
from itertools import filterfalse
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

In [2]:
data=pd.read_csv("emails.csv")
data

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [3]:
counts= data['text'].astype(str).str[:14].value_counts()
counts.to_csv('name.csv')
count=pd.read_csv("name.csv")
count.head(30)

,text,count
0,Subject: re :,1820
1,Subject: inter,67
2,Subject: enron,59
3,Subject: fw :,52
4,Subject: your,48
5,Subject: save,48
6,Subject: reque,40
7,Subject: perfe,32
8,Subject: meeti,30
9,Subject: are y,29


In [4]:
def strip_lines(text):
    if "Subject: re :" in text:
        return text[13:]
    if "Subject: fw :" in text:
        return text[13:]
    if "Subject: fwd :" in text:
        return text[14:]
    if "Subject: news :" in text:
        return text[15:]
    else:
        return text[8:]

In [5]:
data['text']=data['text'].apply(lambda x:strip_lines(x))
data['text']

0        naturally irresistible your corporate identit...
1        the stock trading gunslinger  fanny is merril...
2        unbelievable new homes made easy  im wanting ...
3        4 color printing special  request additional ...
4        do not have money , get software cds from her...
                              ...                        
5723     research and development charges to gpg  here...
5724     receipts from visit  jim ,  thanks again for ...
5725     enron case study update  wow ! all on the sam...
5726     interest  david ,  please , call shirley cren...
5727     aurora 5 . 2 update  aurora version 5 . 2  - ...
Name: text, Length: 5728, dtype: object

In [6]:
nltk.download('punkt')
data['text'] = data['text'].apply(lambda x:word_tokenize(x))
data['text']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0       [naturally, irresistible, your, corporate, ide...
1       [the, stock, trading, gunslinger, fanny, is, m...
2       [unbelievable, new, homes, made, easy, im, wan...
3       [4, color, printing, special, request, additio...
4       [do, not, have, money, ,, get, software, cds, ...
                              ...                        
5723    [research, and, development, charges, to, gpg,...
5724    [receipts, from, visit, jim, ,, thanks, again,...
5725    [enron, case, study, update, wow, !, all, on, ...
5726    [interest, david, ,, please, ,, call, shirley,...
5727    [aurora, 5, ., 2, update, aurora, version, 5, ...
Name: text, Length: 5728, dtype: object

In [7]:
def normalize_tokens(list_of_tokens):
    return map(lambda x:x.lower(),list_of_tokens)

In [8]:
data['text'] = data['text'].apply(lambda x:normalize_tokens(x))

In [9]:
data['text'] = data['text'].apply(lambda x: list(x))
data['text']

0       [naturally, irresistible, your, corporate, ide...
1       [the, stock, trading, gunslinger, fanny, is, m...
2       [unbelievable, new, homes, made, easy, im, wan...
3       [4, color, printing, special, request, additio...
4       [do, not, have, money, ,, get, software, cds, ...
                              ...                        
5723    [research, and, development, charges, to, gpg,...
5724    [receipts, from, visit, jim, ,, thanks, again,...
5725    [enron, case, study, update, wow, !, all, on, ...
5726    [interest, david, ,, please, ,, call, shirley,...
5727    [aurora, 5, ., 2, update, aurora, version, 5, ...
Name: text, Length: 5728, dtype: object

In [10]:
contractions_dict.keys()

dict_keys(["I'm", "I'm'a", "I'm'o", "I've", "I'll", "I'll've", "I'd", "I'd've", 'Whatcha', "amn't", "ain't", "aren't", "'cause", "can't", "can't've", "could've", "couldn't", "couldn't've", "daren't", "daresn't", "dasn't", "didn't", 'didn’t', "don't", 'don’t', "doesn't", "e'er", "everyone's", 'finna', 'gimme', "gon't", 'gonna', 'gotta', "hadn't", "hadn't've", "hasn't", "haven't", "he've", "he's", "he'll", "he'll've", "he'd", "he'd've", "here's", "how're", "how'd", "how'd'y", "how's", "how'll", "isn't", "it's", "'tis", "'twas", "it'll", "it'll've", "it'd", "it'd've", 'kinda', "let's", 'luv', "ma'am", "may've", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've", "ne'er", "o'", "o'clock", "ol'", "oughtn't", "oughtn't've", "o'er", "shan't", "sha'n't", "shalln't", "shan't've", "she's", "she'll", "she'd", "she'd've", "should've", "shouldn't", "shouldn't've", "so've", "so's", "somebody's", "someone's", "something's", 'sux', "that're", "

In [11]:
def contracted_word_expansion(token):
    if token in contractions_dict.keys():
        return contractions_dict[token]
    else:
        return token

In [12]:
def contractions_expansion(list_of_tokens):
    return map(contracted_word_expansion,list_of_tokens)

In [13]:
data['text'] = data['text'].apply(lambda x: contractions_expansion(x))

In [14]:
data['text'] = data['text'].apply(lambda x: list(x))

In [15]:
data['text']

0       [naturally, irresistible, your, corporate, ide...
1       [the, stock, trading, gunslinger, fanny, is, m...
2       [unbelievable, new, homes, made, easy, im, wan...
3       [4, color, printing, special, request, additio...
4       [do, not, have, money, ,, get, software, cds, ...
                              ...                        
5723    [research, and, development, charges, to, gpg,...
5724    [receipts, from, visit, jim, ,, thanks, again,...
5725    [enron, case, study, update, wow, !, all, on, ...
5726    [interest, david, ,, please, ,, call, shirley,...
5727    [aurora, 5, ., 2, update, aurora, version, 5, ...
Name: text, Length: 5728, dtype: object

In [16]:
regex = r'^@[a-zA-z0-9]|^#[a-zA-Z0-9]|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [17]:
def waste_word_or_not(token):
    return re.search(regex,token)

In [18]:
def filter_waste_words(list_of_tokens):
    return filterfalse(waste_word_or_not,list_of_tokens)

In [19]:
data['text'] = data['text'].apply(lambda x: filter_waste_words(x))

In [20]:
data['text'] = data['text'].apply(lambda x: list(x))
data['text']

0       [naturally, irresistible, your, corporate, ide...
1       [the, stock, trading, gunslinger, fanny, is, m...
2       [unbelievable, new, homes, made, easy, im, wan...
3       [color, printing, special, request, additional...
4       [do, not, have, money, get, software, cds, fro...
                              ...                        
5723    [research, and, development, charges, to, gpg,...
5724    [receipts, from, visit, jim, thanks, again, fo...
5725    [enron, case, study, update, wow, all, on, the...
5726    [interest, david, please, call, shirley, crens...
5727    [aurora, update, aurora, version, the, fastest...
Name: text, Length: 5728, dtype: object

In [21]:
def split(list_of_tokens):
    return map(lambda x:re.split(regex,x)[0],list_of_tokens)

In [22]:
data['text'] = data['text'].apply(lambda x: split(x))

In [23]:
data['text'] = data['text'].apply(lambda x: list(x))
data['text']

0       [naturally, irresistible, your, corporate, ide...
1       [the, stock, trading, gunslinger, fanny, is, m...
2       [unbelievable, new, homes, made, easy, im, wan...
3       [color, printing, special, request, additional...
4       [do, not, have, money, get, software, cds, fro...
                              ...                        
5723    [research, and, development, charges, to, gpg,...
5724    [receipts, from, visit, jim, thanks, again, fo...
5725    [enron, case, study, update, wow, all, on, the...
5726    [interest, david, please, call, shirley, crens...
5727    [aurora, update, aurora, version, the, fastest...
Name: text, Length: 5728, dtype: object

In [24]:
nltk.download('stopwords')
en_stop_words=list(set(stopwords.words('english')).union(set(STOP_WORDS)))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def is_stopword(token):
    return not(token in en_stop_words or re.search(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',token))

In [26]:
def stopwords_removal(list_of_tokens):
    return filter(is_stopword,list_of_tokens)

In [27]:
data['text'] = data['text'].apply(lambda x: stopwords_removal(x))

In [28]:
data['text'] = data['text'].apply(lambda x: list(x))
data['text']

0       [naturally, irresistible, corporate, identity,...
1       [stock, trading, gunslinger, fanny, merrill, m...
2       [unbelievable, new, homes, easy, im, wanting, ...
3       [color, printing, special, request, additional...
4       [money, software, cds, software, compatibility...
                              ...                        
5723    [research, development, charges, gpg, forwarde...
5724    [receipts, visit, jim, thanks, invitation, vis...
5725    [enron, case, study, update, wow, day, super, ...
5726    [interest, david, shirley, crenshaw, assistant...
5727    [aurora, update, aurora, version, fastest, mod...
Name: text, Length: 5728, dtype: object

In [29]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith ('N'):
        return(treebank_tag[0],wordnet.NOUN)
    elif treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wordnet.ADJ)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wordnet.ADV)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wordnet.VERB)
    else: 
        return (treebank_tag[0],wordnet.NOUN)

In [30]:
def get_pos_tag(list_of_tokens):
    return map(get_wnet_pos_tag,pos_tag(list_of_tokens))

In [31]:
nltk.download('averaged_perceptron_tagger')
data['text'] = data['text'].apply(lambda x: get_pos_tag(x))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [32]:
nltk.download('wordnet')
data['text'] = data['text'].apply(lambda x: list(x))
data['text']

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0       [(naturally, r), (irresistible, a), (corporate...
1       [(stock, n), (trading, n), (gunslinger, n), (f...
2       [(unbelievable, a), (new, a), (homes, n), (eas...
3       [(color, n), (printing, v), (special, a), (req...
4       [(money, n), (software, n), (cds, n), (softwar...
                              ...                        
5723    [(research, n), (development, n), (charges, n)...
5724    [(receipts, n), (visit, v), (jim, a), (thanks,...
5725    [(enron, n), (case, n), (study, n), (update, a...
5726    [(interest, n), (david, n), (shirley, n), (cre...
5727    [(aurora, a), (update, a), (aurora, n), (versi...
Name: text, Length: 5728, dtype: object

In [33]:
lemmatizer=WordNetLemmatizer()

In [34]:
def token_lemmatization(token_pos_tuple):
    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [35]:
def lemmatization(list_of_tokens):
    if len(list_of_tokens)>0:
        return map(lambda x: token_lemmatization(x),list_of_tokens)

In [36]:
data['text']=data['text'].apply(lambda x:lemmatization(x))

In [37]:
data['text']=data['text'].apply(lambda x: list(x))

In [38]:
data['text']

0       [naturally, irresistible, corporate, identity,...
1       [stock, trading, gunslinger, fanny, merrill, m...
2       [unbelievable, new, home, easy, im, want, home...
3       [color, print, special, request, additional, i...
4       [money, software, cd, software, compatibility,...
                              ...                        
5723    [research, development, charge, gpg, forward, ...
5724    [receipt, visit, jim, thanks, invitation, visi...
5725    [enron, case, study, update, wow, day, super, ...
5726    [interest, david, shirley, crenshaw, assistant...
5727    [aurora, update, aurora, version, fast, model,...
Name: text, Length: 5728, dtype: object

In [39]:
vocab=set()
for list_of_tokens in data['text']:
    vocab=vocab.union(set(list_of_tokens))

In [40]:
vocab=list(vocab)
len(vocab)

29610

In [41]:
data['text'][5727]

['aurora',
 'update',
 'aurora',
 'version',
 'fast',
 'model',
 'get',
 'faster',
 'epis',
 'announces',
 'release',
 'aurora',
 'version',
 'aurora',
 'electric',
 'market',
 'price',
 'forecast',
 'tool',
 'legendary',
 'power',
 'speed',
 'combine',
 'powerful',
 'chronological',
 'dispatch',
 'model',
 'capability',
 'simulate',
 'market',
 'day',
 'year',
 'add',
 'risk',
 'analysis',
 'section',
 'power',
 'user',
 'selectable',
 'monte',
 'carlo',
 'latin',
 'hypercube',
 'model',
 'portfolio',
 'analysis',
 'power',
 'tough',
 'critic',
 'input',
 'output',
 'standard',
 'excel',
 'access',
 'table',
 'get',
 'powerful',
 'tool',
 'market',
 'month',
 'ago',
 'expand',
 'emission',
 'model',
 'capability',
 'add',
 'quarterly',
 'database',
 'update',
 'increase',
 'speed',
 'entire',
 'model',
 'operation',
 'include',
 'new',
 'reporting',
 'enhancement',
 'marginal',
 'reporting',
 'fuel',
 'resource',
 'group',
 'resource',
 'second',
 'ability',
 'display',
 'resource',
 

In [42]:
len(data['text'][5727])

200

In [43]:
data['text'][5727].count('aurora')

9

In [44]:
term_frequency=data['text'][5727].count('aurora')/len(data['text'][5727])
term_frequency
#similarly we will find term frequency for every word in vocab for each mail.

0.045

In [45]:
len(data['text'])

5728

In [46]:
word = 'aurora'
c=0
for row_index in range(0,5727):
 if word in data['text'][row_index]:
    c=c+1
print(c)
total_number_of_rows_containing_word_aurora=c

4


In [47]:
Inverve_document_frequency_idf=total_number_of_rows_containing_word_aurora/len(data['text'])
Inverve_document_frequency_idf
#similarly we will find idf for every word in vocab for each mail.
#a matrix having term_frequency (5727*29610) will get multiplied by idf(1*29610)

0.0006983240223463687

In [48]:
vocab

['abondantly',
 'musharaka',
 'at',
 'uncomofortable',
 'blackrock',
 'autoworker',
 'kwote',
 'aggregated',
 'con',
 'perk',
 'town',
 'dribble',
 'sympathy',
 'ech',
 'kyran',
 'hedgehog',
 'ieader',
 'gainsville',
 'indicative',
 'civilian',
 'rag',
 'exhale',
 'knee',
 'piease',
 'pageable',
 'phenomenology',
 'mayorder',
 'bburn',
 'persuasiveness',
 'jak',
 'docieraly',
 'billow',
 'fish',
 'shoudl',
 'anselm',
 'site',
 'mariadacosta',
 'energywriter',
 'stay',
 'jpeg',
 'wedel',
 'nastepna',
 'bruzzese',
 'sounder',
 'strategic',
 'invstment',
 'asthe',
 'hump',
 'numbness',
 'ekonomicznych',
 'rutkowski',
 'ufl',
 'uklinux',
 'doktora',
 'locust',
 'clifford',
 'itm',
 'intriguing',
 'fluorogenic',
 'duman',
 'sanchez',
 'backroom',
 'allude',
 'href',
 'ste',
 'varied',
 'ww',
 'brand',
 'wrapping',
 'tinubu',
 'vanderbilt',
 'append',
 'globally',
 'alcar',
 'stinson',
 'costantino',
 'abargain',
 'philosophical',
 'ace',
 'grave',
 'principie',
 'neeraj',
 'hebert',
 'philo

In [49]:
vocab_dict = dict(zip(vocab,list(range(0,len(vocab)))))
vocab_dict

{'abondantly': 0,
 'musharaka': 1,
 'at': 2,
 'uncomofortable': 3,
 'blackrock': 4,
 'autoworker': 5,
 'kwote': 6,
 'aggregated': 7,
 'con': 8,
 'perk': 9,
 'town': 10,
 'dribble': 11,
 'sympathy': 12,
 'ech': 13,
 'kyran': 14,
 'hedgehog': 15,
 'ieader': 16,
 'gainsville': 17,
 'indicative': 18,
 'civilian': 19,
 'rag': 20,
 'exhale': 21,
 'knee': 22,
 'piease': 23,
 'pageable': 24,
 'phenomenology': 25,
 'mayorder': 26,
 'bburn': 27,
 'persuasiveness': 28,
 'jak': 29,
 'docieraly': 30,
 'billow': 31,
 'fish': 32,
 'shoudl': 33,
 'anselm': 34,
 'site': 35,
 'mariadacosta': 36,
 'energywriter': 37,
 'stay': 38,
 'jpeg': 39,
 'wedel': 40,
 'nastepna': 41,
 'bruzzese': 42,
 'sounder': 43,
 'strategic': 44,
 'invstment': 45,
 'asthe': 46,
 'hump': 47,
 'numbness': 48,
 'ekonomicznych': 49,
 'rutkowski': 50,
 'ufl': 51,
 'uklinux': 52,
 'doktora': 53,
 'locust': 54,
 'clifford': 55,
 'itm': 56,
 'intriguing': 57,
 'fluorogenic': 58,
 'duman': 59,
 'sanchez': 60,
 'backroom': 61,
 'allude':

In [50]:
len(vocab)

29610

In [51]:
data['text'][0]

['naturally',
 'irresistible',
 'corporate',
 'identity',
 'lt',
 'hard',
 'recollect',
 'company',
 'market',
 'suqgestions',
 'information',
 'isoverwhelminq',
 'good',
 'catchy',
 'logo',
 'stylish',
 'statlonery',
 'outstanding',
 'website',
 'task',
 'easy',
 'promise',
 'havinq',
 'order',
 'iogo',
 'company',
 'automaticaily',
 'world',
 'ieader',
 'isguite',
 'ciear',
 'good',
 'product',
 'effective',
 'business',
 'organization',
 'practicable',
 'aim',
 'hotat',
 'nowadays',
 'market',
 'promise',
 'marketing',
 'effort',
 'effective',
 'list',
 'clear',
 'benefit',
 'creativeness',
 'hand',
 'original',
 'logo',
 'specially',
 'reflect',
 'distinctive',
 'company',
 'image',
 'convenience',
 'logo',
 'stationery',
 'provide',
 'format',
 'easy',
 'use',
 'content',
 'management',
 'system',
 'letsyou',
 'change',
 'website',
 'content',
 'structure',
 'promptness',
 'logo',
 'draft',
 'business',
 'day',
 'affordability',
 'marketing',
 'break',
 'gap',
 'budget',
 'satisfa

In [52]:
def join_tokens(list_of_tokens):
    return " ".join(list_of_tokens)

In [53]:
data['text']=data['text'].apply(lambda x:join_tokens(x))

In [54]:
data['text'][0]

'naturally irresistible corporate identity lt hard recollect company market suqgestions information isoverwhelminq good catchy logo stylish statlonery outstanding website task easy promise havinq order iogo company automaticaily world ieader isguite ciear good product effective business organization practicable aim hotat nowadays market promise marketing effort effective list clear benefit creativeness hand original logo specially reflect distinctive company image convenience logo stationery provide format easy use content management system letsyou change website content structure promptness logo draft business day affordability marketing break gap budget satisfaction guarantee provide unlimited change extra fee surethat love result collaboration look portfolio interested'

In [55]:
corpus=list()
for email_text in data['text']:
    corpus.append(email_text)


In [56]:
vectorizer=TfidfVectorizer(vocabulary=vocab_dict)
tf_idf_matrix=vectorizer.fit_transform(corpus)
tf_idf_matrix

<5728x29610 sparse matrix of type '<class 'numpy.float64'>'
	with 438210 stored elements in Compressed Sparse Row format>

In [57]:
tf_idf_matrix=tf_idf_matrix.toarray()
tf_idf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
df=pd.DataFrame(tf_idf_matrix)
df

,0,1,2,3,4,5,6,7,8,9,...,29600,29601,29602,29603,29604,29605,29606,29607,29608,29609
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
df['spam'] = data['spam']

In [60]:
pca=PCA(n_components=5000)

In [61]:
tf_idf_matrix_reduced=pca.fit_transform(tf_idf_matrix)

In [62]:
df=pd.DataFrame(tf_idf_matrix_reduced)

In [63]:
df['spam'] = data['spam']

In [64]:
gnb = GaussianNB()

In [69]:
X_train=df.iloc[:,0:5000]
y_train=df['spam']

In [70]:
gnb.fit(X=X_train,y=y_train)

GaussianNB()

In [71]:
predicted_categories = gnb.predict(X_train)

In [72]:
print(classification_report(y_true=y_train,y_pred=predicted_categories))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      4360
           1       0.52      0.51      0.51      1368

    accuracy                           0.77      5728
   macro avg       0.68      0.68      0.68      5728
weighted avg       0.77      0.77      0.77      5728

